In [1]:
import json
from collections import defaultdict
import os

# ✅ Path to your JSON file in Kaggle (adjust if filename differs)
FILE_PATH = "/kaggle/input/user-wallet-transactions-json/user-wallet-transactions.json"

def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def explore_data(data):
    action_counts = defaultdict(int)
    sample_by_action = {}
    unique_wallets = set()

    for tx in data:
        action = tx.get('action', 'unknown')
        wallet = tx.get('userWallet')
        action_counts[action] += 1
        unique_wallets.add(wallet)

        # Store first example per action
        if action not in sample_by_action:
            sample_by_action[action] = tx

    # Output summary
    print(f"\n📦 Total transactions: {len(data)}")
    print(f"👛 Unique wallets: {len(unique_wallets)}")
    print(f"\n🔍 Action counts:")
    for action, count in action_counts.items():
        print(f"  - {action}: {count}")

    print("\n📑 Sample transaction per action type:")
    for action, sample in sample_by_action.items():
        print(f"\n=== {action.upper()} ===")
        print(json.dumps(sample, indent=2))

# ✅ Run everything
data = load_data(FILE_PATH)
explore_data(data)



📦 Total transactions: 100000
👛 Unique wallets: 3497

🔍 Action counts:
  - deposit: 37808
  - redeemunderlying: 32305
  - borrow: 17086
  - repay: 12553
  - liquidationcall: 248

📑 Sample transaction per action type:

=== DEPOSIT ===
{
  "_id": {
    "$oid": "681d38fed63812d4655f571a"
  },
  "userWallet": "0x00000000001accfa9cef68cf5371a23025b6d4b6",
  "network": "polygon",
  "protocol": "aave_v2",
  "txHash": "0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190",
  "logId": "0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190_Deposit",
  "timestamp": 1629178166,
  "blockNumber": 1629178166,
  "action": "deposit",
  "actionData": {
    "type": "Deposit",
    "amount": "2000000000",
    "assetSymbol": "USDC",
    "assetPriceUSD": "0.9938318274296357543568636362026045",
    "poolId": "0x2791bca1f2de4661ed88a30c99a7a9449aa84174",
    "userId": "0x00000000001accfa9cef68cf5371a23025b6d4b6"
  },
  "__v": 0,
  "createdAt": {
    "$date": "2025-05-08T23:06:39.465Z

In [3]:
import pandas as pd

df = pd.DataFrame(data)
# Load the JSON file (you've already done this, assuming it's in df)
df['amount'] = df['actionData'].apply(lambda x: float(x.get('amount', 0)))
df['asset_price_usd'] = df['actionData'].apply(lambda x: float(x.get('assetPriceUSD', 0)))
df['amount_usd'] = df['amount'] * df['asset_price_usd']

# Optional: convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# One-hot encode action types
action_dummies = pd.get_dummies(df['action'], prefix='action')
df = pd.concat([df, action_dummies], axis=1)

# Group by userWallet
wallet_summary = df.groupby('userWallet').agg({
    'amount_usd': ['sum'],  # Total USD across all actions
    'action_deposit': 'sum',
    'action_borrow': 'sum',
    'action_repay': 'sum',
    'action_liquidationcall': 'sum',
    'action_redeemunderlying': 'sum'
}).reset_index()

# Flatten column names
wallet_summary.columns = ['userWallet', 'total_amount_usd', 'num_deposits', 'num_borrows', 'num_repays', 'num_liquidations', 'num_withdrawals']

# Recalculate totals by action (in USD)
def sum_amount_for_action(df, action):
    return df[df['action'] == action].groupby('userWallet')['amount_usd'].sum()

deposit_usd = sum_amount_for_action(df, 'deposit')
borrow_usd = sum_amount_for_action(df, 'borrow')
repay_usd = sum_amount_for_action(df, 'repay')

wallet_summary = wallet_summary.set_index('userWallet')
wallet_summary['total_deposit_usd'] = deposit_usd
wallet_summary['total_borrow_usd'] = borrow_usd
wallet_summary['total_repay_usd'] = repay_usd

# Calculate key ratios
wallet_summary['repay_to_borrow_ratio'] = wallet_summary['total_repay_usd'] / (wallet_summary['total_borrow_usd'] + 1e-6)
wallet_summary['borrow_to_deposit_ratio'] = wallet_summary['total_borrow_usd'] / (wallet_summary['total_deposit_usd'] + 1e-6)

# Reset index
wallet_summary = wallet_summary.reset_index()

# Preview the feature set
wallet_summary.head()


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,userWallet,total_amount_usd,num_deposits,num_borrows,num_repays,num_liquidations,num_withdrawals,total_deposit_usd,total_borrow_usd,total_repay_usd,repay_to_borrow_ratio,borrow_to_deposit_ratio
0,0x00000000001accfa9cef68cf5371a23025b6d4b6,1.987664e+09,1,0,0,0,0,1.987664e+09,NaN,NaN,NaN,NaN
1,0x000000000051d07a4fb3bd10121a343d85818da6,2.856945e+20,1,0,0,0,0,2.856945e+20,NaN,NaN,NaN,NaN
2,0x000000000096026fb41fc39f9875d164bd82e2dc,5.152311e+15,2,0,0,0,0,5.152311e+15,NaN,NaN,NaN,NaN
3,0x0000000000e189dd664b9ab08a33c4839953852c,9.803600e+20,0,0,0,0,17,NaN,NaN,NaN,NaN,NaN
4,0x0000000002032370b971dabd36d72f3e5a7bf1ee,3.797495e+23,250,15,4,0,130,2.065773e+23,6.029762e+22,7.015026e+10,1.163400e-12,0.291889


In [20]:
import json
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from collections import defaultdict
import os

# Kaggle file paths
FILE_PATH = "/kaggle/input/user-wallet-transactions-json/user-wallet-transactions.json"
OUTPUT_PATH = "/kaggle/working/wallet_scores2.csv"
PLOT_PATH = "/kaggle/working/score_distribution2.png"

def load_data(file_path):
    """Load JSON transaction data."""
    with open(file_path, 'r') as f:
        return json.load(f)

def explore_data(data):
    """Print data summary and sample transactions by action type."""
    action_counts = defaultdict(int)
    sample_by_action = {}
    unique_wallets = set()

    for tx in data:
        action = tx.get('action', 'unknown')
        wallet = tx.get('userWallet')
        action_counts[action] += 1
        unique_wallets.add(wallet)
        if action not in sample_by_action:
            sample_by_action[action] = tx

    print(f"\n📦 Total transactions: {len(data)}")
    print(f"👛 Unique wallets: {len(unique_wallets)}")
    print(f"\n🔍 Action counts:")
    for action, count in action_counts.items():
        print(f"  - {action}: {count}")
    print("\n📑 Sample transaction per action type:")
    for action, sample in sample_by_action.items():
        print(f"\n=== {action.upper()} ===")
        print(json.dumps(sample, indent=2))

def engineer_features(data):
    """Engineer features from transaction data."""
    df = pd.DataFrame(data)
    
    # Extract amount and USD value
    df['amount'] = df['actionData'].apply(lambda x: float(x.get('amount', 0)))
    df['asset_price_usd'] = df['actionData'].apply(lambda x: float(x.get('assetPriceUSD', 0)))
    df['amount_usd'] = df['amount'] * df['asset_price_usd']
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    
    # One-hot encode actions
    action_dummies = pd.get_dummies(df['action'], prefix='action')
    df = pd.concat([df, action_dummies], axis=1)
    
    # Aggregate by wallet
    wallet_summary = df.groupby('userWallet').agg({
        'amount_usd': ['sum'],
        'action_deposit': 'sum',
        'action_borrow': 'sum',
        'action_repay': 'sum',
        'action_liquidationcall': 'sum',
        'action_redeemunderlying': 'sum'
    }).reset_index()
    
    wallet_summary.columns = ['userWallet', 'total_amount_usd', 'num_deposits', 'num_borrows', 
                             'num_repays', 'num_liquidations', 'num_withdrawals']
    
    # USD sums per action
    wallet_summary = wallet_summary.set_index('userWallet')
    deposit_usd = df[df['action'] == 'deposit'].groupby('userWallet')['amount_usd'].sum()
    borrow_usd = df[df['action'] == 'borrow'].groupby('userWallet')['amount_usd'].sum()
    repay_usd = df[df['action'] == 'repay'].groupby('userWallet')['amount_usd'].sum()
    wallet_summary['total_deposit_usd'] = deposit_usd
    wallet_summary['total_borrow_usd'] = borrow_usd
    wallet_summary['total_repay_usd'] = repay_usd
    
    # Ratios and additional features
    wallet_summary['repay_to_borrow_ratio'] = wallet_summary['total_repay_usd'] / (wallet_summary['total_borrow_usd'] + 1e-6)
    wallet_summary['borrow_to_deposit_ratio'] = wallet_summary['total_borrow_usd'] / (wallet_summary['total_deposit_usd'] + 1e-6)
    wallet_summary = wallet_summary.reset_index()
    wallet_summary['tx_count'] = df.groupby('userWallet').size().reset_index(name='count')['count']
    wallet_summary['unique_assets'] = df.groupby('userWallet')['actionData'].apply(lambda x: len(set(tx.get('assetSymbol') for tx in x))).reset_index(name='count')['count']
    wallet_summary['wallet_age_days'] = (df.groupby('userWallet')['timestamp'].max() - df.groupby('userWallet')['timestamp'].min()).dt.total_seconds().reset_index(name='age')['age'] / (24 * 3600)
    wallet_summary['tx_frequency'] = wallet_summary['tx_count'] / (wallet_summary['wallet_age_days'] + 1e-6)
    wallet_summary['liquidation_ratio'] = wallet_summary['num_liquidations'] / (wallet_summary['tx_count'] + 1e-6)
    
    return wallet_summary.fillna(0)

def compute_heuristic_score(wallet_summary):
    """Compute heuristic score as proxy target."""
    return (
        0.4 * wallet_summary['repay_to_borrow_ratio'].clip(0, 5) +
        0.3 * wallet_summary['num_deposits'] / (wallet_summary['tx_count'] + 1e-6) +
        0.2 * wallet_summary['unique_assets'] / 10 +
        0.1 * np.log1p(wallet_summary['wallet_age_days']) -
        0.5 * wallet_summary['liquidation_ratio']
    )

def train_and_evaluate(wallet_summary):
    """Train XGBoost model and evaluate with train-test split."""
    features = [
        'total_deposit_usd', 'total_borrow_usd', 'total_repay_usd',
        'num_deposits', 'num_borrows', 'num_repays', 'num_liquidations', 
        'num_withdrawals', 'repay_to_borrow_ratio', 'borrow_to_deposit_ratio',
        'liquidation_ratio', 'wallet_age_days', 'tx_frequency', 'unique_assets'
    ]
    
    X = wallet_summary[features]
    y = compute_heuristic_score(wallet_summary)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train XGBoost
    model = XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
    model.fit(X_train, y_train)
    
    # Evaluate on training set
    y_pred_train = model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_pred_train)
    train_rmse = np.sqrt(train_mse)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    train_r2 = r2_score(y_train, y_pred_train)
    
    # Evaluate on test set
    y_pred_test = model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # Print metrics
    print("\nModel Evaluation Metrics:")
    print("Training Set:")
    print(f"  Mean Squared Error (MSE): {train_mse:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {train_rmse:.4f}")
    print(f"  Mean Absolute Error (MAE): {train_mae:.4f}")
    print(f"  R-squared Score: {train_r2:.4f}")
    print("Test Set:")
    print(f"  Mean Squared Error (MSE): {test_mse:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {test_rmse:.4f}")
    print(f"  Mean Absolute Error (MAE): {test_mae:.4f}")
    print(f"  R-squared Score: {test_r2:.4f}")
    
    # Predict scores for all wallets
    raw_scores = model.predict(X)
    scaler = MinMaxScaler(feature_range=(0, 1000))
    scores = scaler.fit_transform(raw_scores.reshape(-1, 1)).flatten()
    
    wallet_summary['credit_score'] = scores
    return wallet_summary[['userWallet', 'credit_score'] + features]

def plot_score_distribution(wallet_summary):
    """Plot and save score distribution histogram."""
    bins = range(0, 1100, 100)
    plt.hist(wallet_summary['credit_score'], bins=bins, edgecolor='black')
    plt.title('Credit Score Distribution')
    plt.xlabel('Credit Score')
    plt.ylabel('Number of Wallets')
    plt.savefig(PLOT_PATH)
    plt.close()

def main():
    """Main function to process data, score wallets, and save outputs."""
    data = load_data(FILE_PATH)
    explore_data(data)
    wallet_summary = engineer_features(data)
    scored_wallets = train_and_evaluate(wallet_summary)
    scored_wallets.to_csv(OUTPUT_PATH, index=False)
    print(f"Saved scores to {OUTPUT_PATH}")
    plot_score_distribution(scored_wallets)
    print(f"Saved plot to {PLOT_PATH}")

if __name__ == "__main__":
    main()


📦 Total transactions: 100000
👛 Unique wallets: 3497

🔍 Action counts:
  - deposit: 37808
  - redeemunderlying: 32305
  - borrow: 17086
  - repay: 12553
  - liquidationcall: 248

📑 Sample transaction per action type:

=== DEPOSIT ===
{
  "_id": {
    "$oid": "681d38fed63812d4655f571a"
  },
  "userWallet": "0x00000000001accfa9cef68cf5371a23025b6d4b6",
  "network": "polygon",
  "protocol": "aave_v2",
  "txHash": "0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190",
  "logId": "0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190_Deposit",
  "timestamp": 1629178166,
  "blockNumber": 1629178166,
  "action": "deposit",
  "actionData": {
    "type": "Deposit",
    "amount": "2000000000",
    "assetSymbol": "USDC",
    "assetPriceUSD": "0.9938318274296357543568636362026045",
    "poolId": "0x2791bca1f2de4661ed88a30c99a7a9449aa84174",
    "userId": "0x00000000001accfa9cef68cf5371a23025b6d4b6"
  },
  "__v": 0,
  "createdAt": {
    "$date": "2025-05-08T23:06:39.465Z